In [1]:
# Import our dependencies
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [2]:
# Import csv
from google.colab import files
my_file = files.upload()
df = pd.read_csv("charity_data.csv")

Saving charity_data.csv to charity_data.csv


In [3]:
df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [4]:
# view column names for copy/paste use
df.columns

Index(['EIN', 'NAME', 'APPLICATION_TYPE', 'AFFILIATION', 'CLASSIFICATION',
       'USE_CASE', 'ORGANIZATION', 'STATUS', 'INCOME_AMT',
       'SPECIAL_CONSIDERATIONS', 'ASK_AMT', 'IS_SUCCESSFUL'],
      dtype='object')

In [5]:
df["ASK_AMT"].value_counts()

5000        25398
10478           3
15583           3
63981           3
6725            3
            ...  
5371754         1
30060           1
43091152        1
18683           1
36500179        1
Name: ASK_AMT, Length: 8747, dtype: int64

In [6]:
# drope NAME and EIN columns
df=df.drop(columns=["EIN","NAME","STATUS"], axis=1)

In [7]:
# view df to confirm dropped columns
df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,100000-499999,N,142590,1


In [8]:
# create list of dtypes to help determine unique values
df_cat=df.dtypes[df.dtypes == "object"].index.tolist()

In [9]:
# display number of unique values
df[df_cat].nunique()

APPLICATION_TYPE          17
AFFILIATION                6
CLASSIFICATION            71
USE_CASE                   5
ORGANIZATION               4
INCOME_AMT                 9
SPECIAL_CONSIDERATIONS     2
dtype: int64

In [10]:
# see how many unique values in column to see if bucketing is needed
df["APPLICATION_TYPE"].value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [11]:
# set up binning for application type column by normalizing data in column, choose cutoff point of 155 based on above value_counts
frequencies = df["APPLICATION_TYPE"].value_counts(normalize=True, ascending=True)
frequencies

T17    0.000029
T15    0.000058
T29    0.000058
T14    0.000087
T25    0.000087
T2     0.000466
T12    0.000787
T13    0.001924
T9     0.004548
T10    0.015394
T7     0.021138
T8     0.021488
T19    0.031050
T5     0.034199
T6     0.035453
T4     0.044958
T3     0.788274
Name: APPLICATION_TYPE, dtype: float64

In [12]:
# specify threshold
threshold = frequencies[(frequencies.cumsum() > 0.001).idxmax()]

In [13]:
# recreate column within df based on the cutoff point and rename all values that aren't past cutoff as Other
df['APPLICATION_TYPE']=df['APPLICATION_TYPE'].mask(df['APPLICATION_TYPE'].map(df['APPLICATION_TYPE'].value_counts(normalize=True)) < 0.001, 'Other')
df["APPLICATION_TYPE"].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
T9         156
T13         66
Other       54
Name: APPLICATION_TYPE, dtype: int64

In [14]:
# see how many unique values in column to see if bucketing is needed
df["CLASSIFICATION"].value_counts().head(25)

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
Name: CLASSIFICATION, dtype: int64

In [15]:
# set up binning for application type column by normalizing data in column, choose cutoff point of 103 based on above value_counts
frequencies_classification = df["CLASSIFICATION"].value_counts(normalize=True, ascending=True)
frequencies_classification.tail(25)

C6000    0.000437
C1800    0.000437
C1500    0.000466
C7120    0.000525
C8000    0.000583
C1240    0.000875
C2300    0.000933
C7200    0.000933
C1400    0.000991
C1230    0.001050
C1280    0.001458
C1300    0.001691
C7100    0.002187
C2800    0.002770
C2700    0.003032
C1270    0.003324
C5000    0.003382
C4000    0.005656
C1700    0.008368
C7000    0.022654
C2100    0.054900
C3000    0.055920
C1200    0.141025
C2000    0.177090
C1000    0.505146
Name: CLASSIFICATION, dtype: float64

In [16]:
threshold_classification = frequencies_classification[(frequencies_classification.cumsum() > 0.001).idxmax()]

In [17]:
df['CLASSIFICATION']=df['CLASSIFICATION'].mask(df['CLASSIFICATION'].map(df['CLASSIFICATION'].value_counts(normalize=True)) < 0.001, 'Other')
df["CLASSIFICATION"].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
Other      355
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
Name: CLASSIFICATION, dtype: int64

In [18]:
df

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,100000-499999,N,142590,1
...,...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,0,N,5000,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,0,N,5000,0
34296,T3,CompanySponsored,C2000,Preservation,Association,0,N,5000,0
34297,T5,Independent,C3000,ProductDev,Association,0,N,5000,1


In [19]:
#import onehotencoder
from sklearn.preprocessing import OneHotEncoder

# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(df[df_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names_out(df_cat)
encode_df

,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T13,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
34295,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
34296,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
34297,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [20]:
# merge original df and encoded and then drop the categorical columns to leave only numerical columns
df_merge = df.merge(encode_df, left_index=True, right_index=True)
df_merge = df_merge.drop(df_cat, axis='columns')
df_merge.head()

,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T13,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,5000,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,108590,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,5000,0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,6692,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,142590,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [21]:
# define dataframe for X and y
df_pre = df_merge.drop("IS_SUCCESSFUL", axis=1)
df_pre

,ASK_AMT,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T13,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,5000,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,108590,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,5000,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,6692,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,142590,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,5000,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
34295,5000,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
34296,5000,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
34297,5000,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [22]:
# define dataframe for X and y
X = df_pre
y = df["IS_SUCCESSFUL"]

In [23]:
# train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [24]:
# scale the dataset
X_scaler = StandardScaler()
X_scaler.fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [25]:
# OPTIMIZE #1 - REMOVED STATUS COLUMN - TRY 50 EPOCHS

In [26]:
# Define the model - deep neural net
number_input_features = X_train.shape[1]
hidden_nodes_layer1 =  8
hidden_nodes_layer2 = 5

nn1 = tf.keras.models.Sequential()

# First hidden layer
nn1.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn1.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn1.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 8)                 448       
                                                                 
 dense_1 (Dense)             (None, 5)                 45        
                                                                 
 dense_2 (Dense)             (None, 1)                 6         
                                                                 
Total params: 499
Trainable params: 499
Non-trainable params: 0
_________________________________________________________________


In [27]:
 # Compile the model
nn1.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [28]:
# Train the model
fit_model = nn1.fit(X_train_scaled,y_train,epochs=50)

Epoch 1/50
804/804 [==============================] - 3s 2ms/step - loss: 0.6104 - accuracy: 0.6837
Epoch 2/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5615 - accuracy: 0.7264
Epoch 3/50
804/804 [==============================] - 2s 3ms/step - loss: 0.5571 - accuracy: 0.7275
Epoch 4/50
804/804 [==============================] - 3s 3ms/step - loss: 0.5550 - accuracy: 0.7275
Epoch 5/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5513 - accuracy: 0.7287
Epoch 6/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5497 - accuracy: 0.7296
Epoch 7/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5491 - accuracy: 0.7295
Epoch 8/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5484 - accuracy: 0.7308
Epoch 9/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5478 - accuracy: 0.7318
Epoch 10/50
804/804 [==============================] - 2s 3ms/step - loss: 0.5473 - accuracy: 0.7316

In [29]:
 # Evaluate the model using the test data
model_loss, model_accuracy = nn1.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5520 - accuracy: 0.7301 - 512ms/epoch - 2ms/step
Loss: 0.5519545674324036, Accuracy: 0.7301457524299622


In [30]:
# The original accuracy was 72.96% with a loss of 55.16% compared to the above accuracy and loss of 73.10% and 55.26% respectively.
# This optimization test performed slightly better than the original model in the initial_AlphabetSoupCharity-deep-learning-analysis.ipynb

In [31]:
# OPTIMIZE #2 - REMOVED STATUS COLUMN - TRY 200 EPOCHS

In [32]:
# Define the model - deep neural net
number_input_features = X_train.shape[1]
hidden_nodes_layer1 =  8
hidden_nodes_layer2 = 5

nn2 = tf.keras.models.Sequential()

# First hidden layer
nn2.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn2.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn2.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 8)                 448       
                                                                 
 dense_4 (Dense)             (None, 5)                 45        
                                                                 
 dense_5 (Dense)             (None, 1)                 6         
                                                                 
Total params: 499
Trainable params: 499
Non-trainable params: 0
_________________________________________________________________


In [33]:
 # Compile the model
nn2.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [35]:
 # Train the model
fit_model = nn2.fit(X_train_scaled,y_train,epochs=200)

Epoch 1/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5385 - accuracy: 0.7373
Epoch 2/200
804/804 [==============================] - 2s 3ms/step - loss: 0.5383 - accuracy: 0.7388
Epoch 3/200
804/804 [==============================] - 3s 3ms/step - loss: 0.5384 - accuracy: 0.7385
Epoch 4/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5383 - accuracy: 0.7376
Epoch 5/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5384 - accuracy: 0.7369
Epoch 6/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5384 - accuracy: 0.7383
Epoch 7/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5383 - accuracy: 0.7379
Epoch 8/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5385 - accuracy: 0.7382
Epoch 9/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5382 - accuracy: 0.7387
Epoch 10/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5384 - accura

In [36]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn2.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5489 - accuracy: 0.7283 - 734ms/epoch - 3ms/step
Loss: 0.5489427447319031, Accuracy: 0.7282798886299133


In [37]:
# The original accuracy was 72.96% with a loss of 55.16% compared to the above accuracy and loss of 73.06% and 55.42% respectively.
# This optimization test performed slightly betterthan the original model in the initial_AlphabetSoupCharity-deep-learning-analysis.ipynb
# This performed slightly worse than the 50 epoch test.

In [38]:
# OPTIMIZE #3 - REMOVED STATUS COLUMN - TRY MORE NEURONS IN THE 2 HIDDEN LAYERS

In [39]:
# Define the model - deep neural net
number_input_features = X_train.shape[1]
hidden_nodes_layer1 =  16
hidden_nodes_layer2 = 10

nn3 = tf.keras.models.Sequential()

# First hidden layer
nn3.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn3.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn3.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn3.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 16)                896       
                                                                 
 dense_7 (Dense)             (None, 10)                170       
                                                                 
 dense_8 (Dense)             (None, 1)                 11        
                                                                 
Total params: 1,077
Trainable params: 1,077
Non-trainable params: 0
_________________________________________________________________


In [40]:
 # Compile the model
nn3.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [41]:
# Train the model
fit_model = nn3.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5935 - accuracy: 0.7095
Epoch 2/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5571 - accuracy: 0.7275
Epoch 3/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5526 - accuracy: 0.7302
Epoch 4/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5502 - accuracy: 0.7313
Epoch 5/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5485 - accuracy: 0.7321
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5474 - accuracy: 0.7325
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5469 - accuracy: 0.7328
Epoch 8/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5466 - accuracy: 0.7327
Epoch 9/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5459 - accuracy: 0.7338
Epoch 10/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5453 - accura

In [42]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn3.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5521 - accuracy: 0.7290 - 460ms/epoch - 2ms/step
Loss: 0.5520553588867188, Accuracy: 0.7289795875549316


In [43]:
# The original accuracy was 72.96% with a loss of 55.16% compared to the above accuracy and loss of 72.98% and 54.96% respectively.
# This optimization test performed slightly better than the original model in the initial_AlphabetSoupCharity-deep-learning-analysis.ipynb

In [44]:
# OPTIMIZE #4 - REMOVED STATUS COLUMN - TRY ADDING A HIDDEN LAYER WITH MORE NEURONS IN LAYER 1 and 2 
# ALSO INCLUDE 50 EPOCHS INSTEAD OF 100 SINCE IT PERFORMED BETTER

In [45]:
# Define the model - deep neural net
number_input_features = X_train.shape[1]
hidden_nodes_layer1 =  8
hidden_nodes_layer2 = 5
hidden_nodes_layer3 = 3

nn4 = tf.keras.models.Sequential()

# First hidden layer
nn4.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn4.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn4.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Output layer
nn4.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn4.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_9 (Dense)             (None, 8)                 448       
                                                                 
 dense_10 (Dense)            (None, 5)                 45        
                                                                 
 dense_11 (Dense)            (None, 3)                 18        
                                                                 
 dense_12 (Dense)            (None, 1)                 4         
                                                                 
Total params: 515
Trainable params: 515
Non-trainable params: 0
_________________________________________________________________


In [46]:
# Compile the model
nn4.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [48]:
# Train the model
fit_model = nn4.fit(X_train_scaled,y_train,epochs=50)

Epoch 1/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5415 - accuracy: 0.7337
Epoch 2/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5418 - accuracy: 0.7332
Epoch 3/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5414 - accuracy: 0.7338
Epoch 4/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5416 - accuracy: 0.7330
Epoch 5/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5415 - accuracy: 0.7340
Epoch 6/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5415 - accuracy: 0.7326
Epoch 7/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5417 - accuracy: 0.7338
Epoch 8/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5416 - accuracy: 0.7344
Epoch 9/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5414 - accuracy: 0.7322
Epoch 10/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5416 - accuracy: 0.7340

In [49]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn4.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5504 - accuracy: 0.7277 - 487ms/epoch - 2ms/step
Loss: 0.5503966808319092, Accuracy: 0.7276967763900757


In [50]:
# The original accuracy was 72.96% with a loss of 55.16% compared to the above accuracy and loss of 72.97% and 55.46% respectively.
# This optimization test performed slighly better than the original model in the initial_AlphabetSoupCharity-deep-learning-analysis.ipynb

In [51]:
# NONE OF THE TESTS REACHED 75%

In [57]:
nn1.save("AlphabetSoupCharity_Optimization_1.h5")




In [58]:
nn2.save("AlphabetSoupCharity_Optimization_2.h5")


In [59]:
nn3.save("AlphabetSoupCharity_Optimization_3.h5")

In [60]:
nn4.save("AlphabetSoupCharity_Optimization_4.h5")

In [61]:
from google.colab import files
files.download("AlphabetSoupCharity_Optimization_1.h5")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [62]:
from google.colab import files

files.download("AlphabetSoupCharity_Optimization_2.h5")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [63]:
from google.colab import files

files.download("AlphabetSoupCharity_Optimization_3.h5")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [64]:
from google.colab import files

files.download("AlphabetSoupCharity_Optimization_4.h5")



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>